<a href="https://colab.research.google.com/github/Ashara-kosi/customerResponse/blob/main/Customer_Response_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import string 
import nltk

In [2]:
tweets = pd.read_csv('/content/drive/MyDrive/cleaned_text.csv')
tweets.head()

,Unnamed: 0,new_text
0,0,understand would like assist would need get pr...
1,2,send several private message one respond usual
2,3,please send -PRON- private message assist clic...
3,4,NaN
4,5,please send -PRON- private message gain detail...


In [3]:
tweets.isnull().sum()

Unnamed: 0        0
new_text      18327
dtype: int64

In [4]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 976810 entries, 0 to 976809
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  976810 non-null  int64 
 1   new_text    958483 non-null  object
dtypes: int64(1), object(1)
memory usage: 14.9+ MB


In [5]:
#removing null values
tweets = tweets.dropna()

In [6]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
#Sentiment Analysis
SIA = SentimentIntensityAnalyzer()
tweets["new_text"]= tweets["new_text"].astype(str)
# Applying Model, Variable Creation
tweets['Polarity Score']=tweets["new_text"].apply(lambda x:SIA.polarity_scores(x)['compound'])
tweets['Neutral Score']=tweets["new_text"].apply(lambda x:SIA.polarity_scores(x)['neu'])
tweets['Negative Score']=tweets["new_text"].apply(lambda x:SIA.polarity_scores(x)['neg'])
tweets['Positive Score']=tweets["new_text"].apply(lambda x:SIA.polarity_scores(x)['pos'])
# Converting 0 to 1 Decimal Score to a Categorical Variable
tweets['Sentiment']=''
tweets.loc[tweets['Polarity Score']>0,'Sentiment']='Positive'
tweets.loc[tweets['Polarity Score']==0,'Sentiment']='Neutral'
tweets.loc[tweets['Polarity Score']<0,'Sentiment']='Negative'

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [20]:
data = tweets[['Sentiment','new_text']]


In [21]:
data2 = data.loc[data["Sentiment"] == "Neutral" ]

In [22]:
data.drop(data[data['Sentiment'] == 'Neutral'].index, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
data['Sentiment'].replace(to_replace={'Positive':0,'Negative':1},inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [24]:
#seperating the dataset into dependent and independent variables
y = data.iloc[:5000,0]


In [25]:
x = data.iloc[:5000,1]


In [48]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,ComplementNB
from sklearn.metrics import confusion_matrix,accuracy_score

In [51]:
#using count vectorizer to convert the text to matrix
vectorizer = CountVectorizer(lowercase=False)
x_vec = vectorizer.fit_transform(x)


In [52]:
#converting the sparse matrix to dense
X_vec = x_vec.todense()


In [53]:
tfidf = TfidfTransformer() # by default applies "l2" normalization
X_tfidf = tfidf.fit_transform(X_vec)
X_tfidf = X_tfidf.todense()
X_tfidf

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [54]:
#splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

In [55]:
#testing the complementnb model
clf = ComplementNB()
clf.fit(X_train, y_train)

ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False)

In [56]:
y_pred = clf.predict(X_test)

confusion_matrix(y_test, y_pred)

array([[632, 101],
       [ 80, 187]])

In [57]:
print("Number of mislabeled points out of a total %d points : %d"
...       % (X_test.shape[0], (y_test != y_pred).sum()))
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))


Number of mislabeled points out of a total 1000 points : 181

Accuracy: 0.82



In [58]:
#splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, 
                                                    test_size = 0.2, 
                                                    random_state = 0)


In [59]:
#testing the multinomialnb model
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [60]:

y_pred = clf.predict(X_test)

confusion_matrix(y_test, y_pred)

array([[732,   1],
       [243,  24]])

In [61]:
print("Number of mislabeled points out of a total %d points : %d"
...       % (X_test.shape[0], (y_test != y_pred).sum()))
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))



Number of mislabeled points out of a total 1000 points : 244

Accuracy: 0.76

